Computer vision

In [2]:
import cv2   #pip install opencv-contrib-python

In [12]:
video=cv2.VideoCapture('sample.mp4')

In [13]:
while True:
    state,frame=video.read()
    if not state:
        break
    cv2.imshow('video output',frame)
    if cv2.waitKey(0)==27:
        break
video.release()
cv2.destroyAllWindows()